# Three Steps Analysis

In [ ]:
import json
import glob, os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# ---------------------------
# Helper Functions
# ---------------------------
def compute_se(series):
    """Compute the standard error on a pandas Series."""
    n = series.count()
    return series.std(ddof=1) / np.sqrt(n) if n > 1 else 0

def compute_metrics(df, stage):
    """
    Given a DataFrame of interactions (with a column for the stage and a column for ground_truth),
    compute:
      - Total accuracy (unsure responses, coded as 2, count as incorrect)
      - Accuracy on non-unsure responses ("NotUnsureAcc")
      - Unsure rate
      - Utility (0 if unsure; 1 if correct; -1 if incorrect)
    Returns also the computed utility as a Series.
    """
    preds = df[stage]
    # Total accuracy: count unsure as incorrect.
    correct_all = ((preds != 2) & (preds == df['ground_truth'])).astype(float)
    total_acc_mean = correct_all.mean()
    total_acc_se = compute_se(correct_all)
    
    # Accuracy on non-unsure responses.
    non_unsure = preds != 2
    if non_unsure.sum() > 0:
        correct_non_unsure = (df.loc[non_unsure, stage] == df.loc[non_unsure, 'ground_truth']).astype(float)
        acc_mean = correct_non_unsure.mean()
        acc_se = compute_se(correct_non_unsure)
    else:
        acc_mean, acc_se = np.nan, np.nan

    # Unsure rate.
    unsure_indicator = (preds == 2).astype(float)
    unsure_mean = unsure_indicator.mean()
    unsure_se = compute_se(unsure_indicator)
    
    # Utility: 0 if unsure; else 1 if correct, -1 if incorrect.
    # (i.e., 1 - 2*abs(pred - ground_truth))
    def calc_util(row):
        p = row[stage]
        gt = row['ground_truth']
        return 0 if p == 2 else 1 - 2*abs(p - gt)
    utility = df.apply(calc_util, axis=1)
    util_mean = utility.mean()
    util_se = compute_se(utility)
    
    return total_acc_mean, total_acc_se, acc_mean, acc_se, unsure_mean, unsure_se, util_mean, util_se, utility

# ---------------------------
# Settings and Directories
# ---------------------------
settings = [
    'vf_numeric_llava1.5_with_image_q20_i10_s0',
    'contr_numeric_llava1.5_with_image_q20_i10_s0',
    'vf_descriptive_llava1.5_with_image_q20_i10_s0',
    'contr_descriptive_llava1.5_with_image_q20_i10_s0',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0',
    'showbothmetrics_llava1.5_with_image_q20_i10_s0',
    'showbothmetrics_descriptive_llava1.5_with_image_q20_i10_s0',
    'vf_numeric_qwen2.5_vizwiz_q10_i10_s0',
    'vf_descriptive_qwen2.5_vizwiz_q10_i10_s0',
    'combined12_llava1.5_with_image_q20_i10_s0',
    'combined123_llava1.5_with_image_q20_i10_s0',
]
USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/"

# To store scores and confusion matrices across settings
all_setting_scores = []
all_confusion_matrices = []
consistent_choice_threshold = 0.9  # 90% threshold
df_map = {}

# ---------------------------
# Process Each Setting via DataFrame
# ---------------------------
for setting in settings:
    print(f"\nProcessing setting: {setting}")
    pattern = os.path.join(USER_STUDIES_DIR, "prolific_batches", "batch_interaction_data", setting, "*.json")
    files = glob.glob(pattern)
    
    # Create a list of records (each record is an interaction)
    records = []
    for file in files:
        batch_id = os.path.basename(file).split('.')[0]
        with open(file) as f:
            batch_data = json.load(f)
        # Each file contains sessions for multiple users.
        for user_id, session in batch_data.items():
            interactions = session['interactions']
            for interaction in interactions:
                rec = {
                    'user_id': user_id,
                    'batch': batch_id,
                    'question_i': interaction['question_i'],
                    'answeronly': interaction['user_selections'].get('answeronly'),
                    'withexplanation': interaction['user_selections'].get('withexplanation'),
                    'withexplanationquality': interaction['user_selections'].get('withexplanationquality'),
                    # Ground truth: defined as 1 - prediction_is_correct.
                    'ground_truth': 1 - interaction['question']['prediction_is_correct']
                }
                records.append(rec)
    
    # Build DataFrame from the records.
    df = pd.DataFrame(records)
    print(f"Found {df['user_id'].nunique()} users and {len(df)} interactions for setting '{setting}'.")
    
    # ---------------------------
    # Filter Users Based on Consistency
    # ---------------------------
    # For each user, look at all responses across all stages and drop users
    # where one response (0, 1, or 2) comprises ≥90% of their responses.
    def user_is_valid(group):
        responses = pd.concat([group['answeronly'], group['withexplanation'], group['withexplanationquality']])
        if responses.empty:
            return False
        norm_counts = responses.value_counts(normalize=True)
        if norm_counts.max() >= consistent_choice_threshold:
            # If desired, you could print which user is filtered out.
            # print(f"User {group.name} filtered out; response distribution: {norm_counts.to_dict()}")
            return False
        return True

    df_valid = df.groupby('user_id').filter(user_is_valid)

    if df_valid.empty:
        print("No interactions remain after filtering. Skipping evaluation for this setting.")
        continue
        
    # ---------------------------
    # Filter Batches with More Than 30 Annotations
    # ---------------------------
    # For each batch, if it has been annotated more than 30 times (i.e. more than three users,
    # assuming each user annotates about 10 interactions), print the annotation count
    # and then shorten the batch to include only the first 30 annotations.
    batch_counts = df_valid['batch'].value_counts()
    for batch, count in batch_counts.items():
        if count > 30:
            print(f"Batch {batch} has been annotated {count} times, exceeding 30 annotations. Shortening to first 30 annotations.")
    # Group by 'batch' and take only the first 30 entries for each batch
    df_valid = df_valid.groupby('batch').apply(lambda g: g.head(30)).reset_index(drop=True)
    print(f"{df_valid['user_id'].nunique()} users remain after filtering; {len(df_valid)} interactions remain.")
    
    
    # Indicate which batches need to add users
    batch_counts = df_valid['batch'].value_counts()
    for batch, count in batch_counts.items():
        if count % 10: raise ValueError('# Annotation should be divisible by 10, but found not??')
        count = count // 10
        if count > 3:
            raise ValueError(f"After filtering, # users taking the batch should not > 3??")
        diff = 3 - count
        if diff != 0:
            print(f"Batch {batch} need {diff} more users to annotate.")
            
    # Add df to the df_map for later use.
    df_map[setting] = df_valid
    
    
    # ---------------------------
    # Compute Metrics for Each Stage Using DataFrames
    # ---------------------------
    stage_names = ['Answer Only', 'With Explanation', 'With Explanation + Quality']
    stages = ['answeronly', 'withexplanation', 'withexplanationquality']
    metrics = {}
    utilities = {}
    csv_prints = []
    print("\nStage                    \tUnsure Rate\tTotalAcc\tNotUnsureAcc\tUtility")
    for stage, name in zip(stages, stage_names):
        tot_acc, tot_se, acc_mean, acc_se, unsure_mean, unsure_se, util_mean, util_se, util_series = compute_metrics(df_valid, stage)
        metrics[stage] = {
            'total_accuracy_mean': tot_acc,
            'total_accuracy_se': tot_se,
            'accuracy_mean': acc_mean,
            'accuracy_se': acc_se,
            'unsure_rate_mean': unsure_mean,
            'unsure_rate_se': unsure_se,
            'utility_mean': util_mean,
            'utility_se': util_se
        }
        utilities[stage] = util_series
        print(f"{name:<25}\t{unsure_mean*100:5.1f}% ± {unsure_se*100:4.1f}%\t"
              f"{tot_acc*100:5.1f}% ± {tot_se*100:4.1f}%\t"
              f"{acc_mean*100:5.1f}% ± {acc_se*100:4.1f}%\t"
              f"{util_mean:+.3f} ± {util_se:.3f}")
        csv_prints.append(f"{unsure_mean*100:5.1f}% ± {unsure_se*100:4.1f}%, "
              f"{tot_acc*100:5.1f}% ± {tot_se*100:4.1f}%, "
              f"{acc_mean*100:5.1f}% ± {acc_se*100:4.1f}%, "
              f"{util_mean:+.3f} ± {util_se:.3f}")
        
    print(", ".join(csv_prints))

    # ---------------------------
    # Utility Gains Between Stages
    # ---------------------------
    gain_explanation = utilities['withexplanation'] - utilities['answeronly']
    gain_quality = utilities['withexplanationquality'] - utilities['withexplanation']
    expl_gain_mean = gain_explanation.mean()
    expl_gain_se = gain_explanation.std(ddof=1) / np.sqrt(len(gain_explanation)) if len(gain_explanation) > 1 else 0
    qual_gain_mean = gain_quality.mean()
    qual_gain_se = gain_quality.std(ddof=1) / np.sqrt(len(gain_quality)) if len(gain_quality) > 1 else 0
    
    print(f"\nSetting: {setting}")
    print("Stage\t\tUnsure Rate\tUtility over previous stage")
    print(f"Explanation\t {metrics['withexplanation']['unsure_rate_mean']*100:5.1f}%\t\t {expl_gain_mean:+.3f} ± {expl_gain_se:.3f}")
    print(f"Quality\t\t {metrics['withexplanationquality']['unsure_rate_mean']*100:5.1f}%\t\t {qual_gain_mean:+.3f} ± {qual_gain_se:.3f}")
    
    # ---------------------------
    # Compute Confusion Matrix and M1–M4 Scores
    # ---------------------------
    # Compute utility for withexplanation and withexplanationquality directly.
    def compute_utility(val, gt):
        return 0 if val == 2 else 1 - 2*abs(val-gt)
    
    df_valid['util_withexplanation'] = df_valid.apply(lambda row: compute_utility(row['withexplanation'], row['ground_truth']), axis=1)
    df_valid['util_withexplanationquality'] = df_valid.apply(lambda row: compute_utility(row['withexplanationquality'], row['ground_truth']), axis=1)
    
    # For the confusion matrix, map utilities to indices: -1 -> 0, 0 -> 1, 1 -> 2.
    mapping = {-1: 0, 0: 1, 1: 2}
    label_order = [-1, 0, 1]
    label_names = ['Incorrect', 'Unsure', 'Correct']
    
    try:
        cm = np.zeros((3, 3), dtype=int)
        for _, row in df_valid.iterrows():
            u2 = row['util_withexplanation']
            u3 = row['util_withexplanationquality']
            cm[mapping[u2], mapping[u3]] += 1
        cm_transposed = cm.T
    except Exception as e:
        print(f"Error computing confusion matrix or M1–M4 scores: {e}")
        print(f"This would be expected if setting {setting} is combined12 or combined123.")
        continue
        
    print(f"\nSetting: {setting} Step 2 -> Step 3 (transposed)")
    print("\t\t" + "\t".join(label_names))
    for i, label in enumerate(label_names):
        row_values = "\t".join(str(cm_transposed[i, j]) for j in range(3))
        print(f"{label:<10}\t {row_values}")
    
    # Compute M1–M4 using counts from the confusion matrix.
    def count_transitions(val2, val3):
        return ((df_valid['util_withexplanation'] == val2) & (df_valid['util_withexplanationquality'] == val3)).sum()
    
    denom_M1 = count_transitions(0, -1) + count_transitions(0, 0) + count_transitions(0, 1)
    num_M1 = count_transitions(0, -1) + count_transitions(0, 1)
    m1 = num_M1 / denom_M1 if denom_M1 > 0 else np.nan

    denom_M2 = count_transitions(0, -1) + count_transitions(0, 1)
    m2 = count_transitions(0, 1) / denom_M2 if denom_M2 > 0 else np.nan

    denom_M3 = count_transitions(-1, -1) + count_transitions(-1, 0) + count_transitions(-1, 1)
    num_M3 = count_transitions(-1, 0) + count_transitions(-1, 1)
    m3 = num_M3 / denom_M3 if denom_M3 > 0 else np.nan

    denom_M4 = count_transitions(1, -1) + count_transitions(1, 0) + count_transitions(1, 1)
    num_M4 = count_transitions(1, -1) + count_transitions(1, 0)
    m4 = num_M4 / denom_M4 if denom_M4 > 0 else np.nan

    print("\nM1–M4 Scores:")
    print(f"M1 (P(S3 ≠ U | S2 = U)): {m1:.2%}")
    print(f"M2 (P(S3 = C | S2 = U, S3 ≠ U)): {m2:.2%}")
    print(f"M3 (P(S3 ≠ I | S2 = I)): {m3:.2%}")
    print(f"M4 (P(S3 ≠ C | S2 = C)): {m4:.2%}")
    print(f"{m1:.2%}, {m2:.2%}, {m3:.2%}, {m4:.2%}, {m3-m4:.2%}")

    
    # ---------------------------
    # Print Examples Where Step 2 is Correct and Step 3 is Incorrect
    # ---------------------------
    print("\nExamples where Step 2 is correct and Step 3 is incorrect:")
    examples = df_valid[
        (df_valid['withexplanation'] != 2) &
        (df_valid['withexplanation'] == df_valid['ground_truth']) &
        (df_valid['withexplanationquality'] != 2) &
        (df_valid['withexplanationquality'] != df_valid['ground_truth'])
    ]
    if not examples.empty:
        for idx, row in examples.head(5).iterrows():
            print(row.to_json())
    else:
        print("No examples found.")
    
    # ---------------------------
    # Plot: Confusion Matrix and M1–M4 Scores for the Current Setting
    # ---------------------------
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    
    # Plot confusion matrix
    ax = axes[0]
    im = ax.imshow(cm_transposed, cmap='Greens', interpolation='nearest')
    ax.set_title(f"Setting {setting.split('_llava')[0]} Step 2->Step 3\n(WithExplanation -> WithExplanationQuality)")
    ax.set_xticks(np.arange(len(label_names)))
    ax.set_yticks(np.arange(len(label_names)))
    ax.set_xticklabels(label_names)
    ax.set_yticklabels(label_names)
    ax.set_xlabel("WithExplanation")
    ax.set_ylabel("WithExplanationQuality")
    for i in range(len(label_names)):
        for j in range(len(label_names)):
            ax.text(j, i, cm_transposed[i, j], ha="center", va="center", color="black")
    
    # Plot M1–M4 scores as text.
    ax = axes[1]
    ax.axis('off')
    score_text = (f"M1 (P(S3 ≠ U | S2 = U)): {m1:.2%}\n"
                  f"M2 (P(S3 = C | S2 = U, S3 ≠ U)): {m2:.2%}\n"
                  f"M3 (P(S3 ≠ I | S2 = I)): {m3:.2%}\n"
                  f"M4 (P(S3 ≠ C | S2 = C)): {m4:.2%}")
    ax.text(0.5, 0.5, score_text, fontsize=12, ha='center', va='center', transform=ax.transAxes)
    ax.set_title("M1–M4 Scores")
    plt.tight_layout()
    plt.show()
    
    # Save scores and confusion matrix for later comparison/plotting.
    all_setting_scores.append({
        "Setting": setting,
        "Resolution Rate": m1,
        "Positive Conversion Rate": m2,
        "Error Recovery Rate": m3,
        "Correct Loss Rate": m4
    })
    all_confusion_matrices.append((setting, cm_transposed))

# ---------------------------
# Plot Grouped Bar Chart for M1–M4 Scores Across Settings
# ---------------------------
if all_setting_scores:
    df_scores = pd.DataFrame(all_setting_scores)
    df_scores["SettingShort"] = df_scores["Setting"].apply(lambda s: s.split('_llava')[0])
    df_scores["Recovery Gap"] = df_scores["Error Recovery Rate"] - df_scores["Correct Loss Rate"]
    df_scores = df_scores.set_index("SettingShort")
    
    # Transpose so that rows become score types.
    df_scores_T = df_scores[["Resolution Rate", "Positive Conversion Rate", "Error Recovery Rate",
                             "Correct Loss Rate", "Recovery Gap"]].transpose()
    
    ax = df_scores_T.plot(kind='bar', figsize=(12, 6))
    ax.set_ylabel("Score (Fraction)")
    ax.set_title("Scores by Score Type Across Settings")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(title="Setting", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# ---------------------------
# Plot a Grid of Confusion Matrices for Every Setting
# ---------------------------
if all_confusion_matrices:
    n_settings = len(all_confusion_matrices)
    ncols = 2
    nrows = math.ceil(n_settings / ncols)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*5, nrows*5))
    if n_settings > 1:
        axes = np.array(axes).flatten()
    else:
        axes = [axes]
        
    for ax, (setting, cm_transposed) in zip(axes, all_confusion_matrices):
        im = ax.imshow(cm_transposed, cmap='Greens', interpolation='nearest')
        ax.set_title(setting.split('_llava')[0])
        ax.set_xticks(np.arange(len(label_names)))
        ax.set_yticks(np.arange(len(label_names)))
        ax.set_xticklabels(label_names)
        ax.set_yticklabels(label_names)
        ax.set_xlabel("WithExplanation")
        ax.set_ylabel("WithExplanationQuality")
        for i in range(len(label_names)):
            for j in range(len(label_names)):
                ax.text(j, i, cm_transposed[i, j], ha="center", va="center", color="black")
                
    # Turn off any extra axes.
    for ax in axes[len(all_confusion_matrices):]:
        ax.axis('off')
        
    plt.tight_layout()
    plt.show()


In [ ]:
# ---------------------------
# Updated Bar Plot for Resolution Rate and Error Recovery Rate (Separate Plots)
# with Larger Text, Renamed x‑Tick Labels, and Specified Y‑Range for Resolution Rate Plot
# ---------------------------
if all_setting_scores:
    import matplotlib.ticker as mtick
    # Create a DataFrame from the scores and simplify the setting names.
    df_scores = pd.DataFrame(all_setting_scores)
    df_scores["SettingShort"] = df_scores["Setting"].apply(lambda s: s.split('_llava')[0])
    df_scores = df_scores.set_index("SettingShort")
    df_scores.index.name = None  # Remove the index label from the x-axis.
    
    # Define a mapping from the abbreviated setting names to more natural names.
    rename_mapping = {
        'vf_numeric': 'Numeric VF',
        'contr_numeric': 'Numeric CONTR',
        'showbothmetrics': 'Both Metrics (Numeric)',
        'avg_vf_contr_numeric': 'Average VF/CONTR (Numeric)',
        'showbothmetrics_descriptive': 'Both Metrics (Descriptive)',
        'vf_descriptive': 'Descriptive VF',
        'contr_descriptive': 'Descriptive CONTR'
    }
    
    # Extract the two metrics separately.
    resolution = df_scores["Resolution Rate"]
    error_recovery = df_scores["Error Recovery Rate"]
    
    # Set up subplots for each metric.
    fig, axes = plt.subplots(ncols=2, figsize=(12, 7))
    
    # Plot Resolution Rate.
    resolution.plot(kind="bar", ax=axes[0], color="skyblue")
    axes[0].set_title("Resolution Rate Across Settings", fontsize=16, fontweight="bold")
    axes[0].set_ylabel("Resolution Rate (%)", fontsize=16)
    axes[0].grid(axis="y", linestyle="--", alpha=0.7)
    axes[0].set_xlabel("")  # Remove the x-axis label
    axes[0].tick_params(axis="both", labelsize=12)
    # Replace x tick labels using the mapping.
    new_labels = [rename_mapping.get(label.get_text(), label.get_text()) 
                  for label in axes[0].get_xticklabels()]
    axes[0].set_xticklabels(new_labels, rotation=45, ha="right", fontsize=16)
    axes[0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    # Set custom y-axis range
    axes[0].set_ylim(0.6, 1)
    # Annotate each bar (multiplying value by 100 for percentage display).
    for rect in axes[0].patches:
        height = rect.get_height()
        axes[0].text(rect.get_x() + rect.get_width()/2, height, f"{height*100:.1f}%",
                     ha="center", va="bottom", fontsize=14)
    
    # Plot Error Recovery Rate.
    error_recovery.plot(kind="bar", ax=axes[1], color="lightgreen")
    axes[1].set_title("Error Recovery Rate Across Settings", fontsize=16, fontweight="bold")
    axes[1].set_ylabel("Error Recovery Rate (%)", fontsize=16)
    axes[1].grid(axis="y", linestyle="--", alpha=0.7)
    axes[1].set_xlabel("")  # Remove the x-axis label
    axes[1].tick_params(axis="both", labelsize=12)
    new_labels = [rename_mapping.get(label.get_text(), label.get_text()) 
                  for label in axes[1].get_xticklabels()]
    axes[1].set_xticklabels(new_labels, rotation=45, ha="right", fontsize=16)
    axes[1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axes[1].set_ylim(0, 0.35)
    for rect in axes[1].patches:
        height = rect.get_height()
        axes[1].text(rect.get_x() + rect.get_width()/2, height, f"{height*100:.1f}%",
                     ha="center", va="bottom", fontsize=14)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Store df_map to a file for later use.
import pickle
for setting, df in df_map.items():
    # only keep the columns we need
    df_map[setting] = df[['user_id', 'batch', 'question_i', 'answeronly', 'withexplanation', 'withexplanationquality', 'ground_truth']]
with open('df_map.pkl', 'wb') as f:
    pickle.dump(df_map, f)

# Combined123 Analysis

In [ ]:
import json
import glob, os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# ---------------------------
# Helper Functions
# ---------------------------
def compute_se(series):
    """Compute the standard error on a pandas Series."""
    n = series.count()
    return series.std(ddof=1) / np.sqrt(n) if n > 1 else 0

def compute_metrics(df, stage):
    """
    Given a DataFrame of interactions (with a column for the stage and a column for ground_truth),
    compute:
      - Total accuracy (unsure responses, coded as 2, count as incorrect)
      - Accuracy on non-unsure responses ("NotUnsureAcc")
      - Unsure rate
      - Utility (0 if unsure; 1 if correct; -1 if incorrect)
    Returns also the computed utility as a Series.
    """
    preds = df[stage]
    # Total accuracy: count unsure as incorrect.
    correct_all = ((preds != 2) & (preds == df['ground_truth'])).astype(float)
    total_acc_mean = correct_all.mean()
    total_acc_se = compute_se(correct_all)
    
    # Accuracy on non-unsure responses.
    non_unsure = preds != 2
    if non_unsure.sum() > 0:
        correct_non_unsure = (df.loc[non_unsure, stage] == df.loc[non_unsure, 'ground_truth']).astype(float)
        acc_mean = correct_non_unsure.mean()
        acc_se = compute_se(correct_non_unsure)
    else:
        acc_mean, acc_se = np.nan, np.nan

    # Unsure rate.
    unsure_indicator = (preds == 2).astype(float)
    unsure_mean = unsure_indicator.mean()
    unsure_se = compute_se(unsure_indicator)
    
    # Utility: 0 if unsure; else 1 if correct, -1 if incorrect.
    # (i.e., 1 - 2*abs(pred - ground_truth))
    def calc_util(row):
        p = row[stage]
        gt = row['ground_truth']
        return 0 if p == 2 else 1 - 2*abs(p - gt)
    utility = df.apply(calc_util, axis=1)
    util_mean = utility.mean()
    util_se = compute_se(utility)
    
    return total_acc_mean, total_acc_se, acc_mean, acc_se, unsure_mean, unsure_se, util_mean, util_se, utility

# ---------------------------
# Settings and Directories
# ---------------------------
settings = [
    'combined12_llava1.5_aokvqa_better_sampled_q10_i10_s0',
    'prodmetric_llava1.5_aokvqa_better_sampled_q10_i10_s0',
    'vf_numeric_llava1.5_aokvqa_better_sampled_q10_i10_s0',
    'vf_descriptive_llava1.5_aokvqa_better_sampled_q10_i10_s0',
]
USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/"

# To store scores and confusion matrices across settings
all_setting_scores = []
all_confusion_matrices = []
consistent_choice_threshold = 0.9  # 90% threshold
df_map = {}

# ---------------------------
# Process Each Setting via DataFrame
# ---------------------------
for setting in settings:
    print(f"\nProcessing setting: {setting}")
    pattern = os.path.join(USER_STUDIES_DIR, "prolific_batches", "batch_interaction_data", setting, "*.json")
    files = glob.glob(pattern)
    
    # Create a list of records (each record is an interaction)
    records = []
    for file in files:
        batch_id = os.path.basename(file).split('.')[0]
        with open(file) as f:
            batch_data = json.load(f)
        # Each file contains sessions for multiple users.
        for user_id, session in batch_data.items():
            interactions = session['interactions']
            for interaction in interactions:
                rec = {
                    'user_id': user_id,
                    'batch': batch_id,
                    'question_i': interaction['question_i'],
                    'answeronly': interaction['user_selections'].get('answeronly'),
                    'withexplanation': interaction['user_selections'].get('withexplanation'),
                    'withexplanationquality': interaction['user_selections'].get('withexplanationquality'),
                    # Ground truth: defined as 1 - prediction_is_correct.
                    'ground_truth': 1 - interaction['question']['prediction_is_correct']
                }
                records.append(rec)
    
    # Build DataFrame from the records.
    df = pd.DataFrame(records)
    print(f"Found {df['user_id'].nunique()} users and {len(df)} interactions for setting '{setting}'.")
    
    # ---------------------------
    # Filter Users Based on Consistency
    # ---------------------------
    # For each user, look at all responses across all stages and drop users
    # where one response (0, 1, or 2) comprises ≥90% of their responses.
    def user_is_valid(group):
        responses = pd.concat([group['answeronly'], group['withexplanation'], group['withexplanationquality']])
        if responses.empty:
            return False
        norm_counts = responses.value_counts(normalize=True)
        if norm_counts.max() >= consistent_choice_threshold:
            # If desired, you could print which user is filtered out.
            # print(f"User {group.name} filtered out; response distribution: {norm_counts.to_dict()}")
            return False
        return True

    df_valid = df.groupby('user_id').filter(user_is_valid)

    if df_valid.empty:
        print("No interactions remain after filtering. Skipping evaluation for this setting.")
        continue
        
    # ---------------------------
    # Filter Batches with More Than 30 Annotations
    # ---------------------------
    # For each batch, if it has been annotated more than 30 times (i.e. more than three users,
    # assuming each user annotates about 10 interactions), print the annotation count
    # and then shorten the batch to include only the first 30 annotations.
    batch_counts = df_valid['batch'].value_counts()
    for batch, count in batch_counts.items():
        if count > 30:
            print(f"Batch {batch} has been annotated {count} times, exceeding 30 annotations. Shortening to first 30 annotations.")
    # Group by 'batch' and take only the first 30 entries for each batch
    df_valid = df_valid.groupby('batch').apply(lambda g: g.head(30)).reset_index(drop=True)
    print(f"{df_valid['user_id'].nunique()} users remain after filtering; {len(df_valid)} interactions remain.")
    
    
    # Indicate which batches need to add users
    batch_counts = df_valid['batch'].value_counts()
    for batch, count in batch_counts.items():
        if count % 10: raise ValueError('# Annotation should be divisible by 10, but found not??')
        count = count // 10
        if count > 3:
            raise ValueError(f"After filtering, # users taking the batch should not > 3??")
        diff = 3 - count
        if diff != 0:
            print(f"Batch {batch} need {diff} more users to annotate.")
            
    # Add df to the df_map for later use.
    df_map[setting] = df_valid
    
    
    # ---------------------------
    # Compute Metrics for Each Stage Using DataFrames
    # ---------------------------
    stage_names = ['Answer Only', 'With Explanation', 'With Explanation + Quality']
    stages = ['answeronly', 'withexplanation', 'withexplanationquality']
    metrics = {}
    utilities = {}
    csv_prints = []
    print("\nStage                    \tUnsure Rate\tTotalAcc\tNotUnsureAcc\tUtility")
    for stage, name in zip(stages, stage_names):
        tot_acc, tot_se, acc_mean, acc_se, unsure_mean, unsure_se, util_mean, util_se, util_series = compute_metrics(df_valid, stage)
        metrics[stage] = {
            'total_accuracy_mean': tot_acc,
            'total_accuracy_se': tot_se,
            'accuracy_mean': acc_mean,
            'accuracy_se': acc_se,
            'unsure_rate_mean': unsure_mean,
            'unsure_rate_se': unsure_se,
            'utility_mean': util_mean,
            'utility_se': util_se
        }
        utilities[stage] = util_series
        print(f"{name:<25}\t{unsure_mean*100:5.1f}% ± {unsure_se*100:4.1f}%\t"
              f"{tot_acc*100:5.1f}% ± {tot_se*100:4.1f}%\t"
              f"{acc_mean*100:5.1f}% ± {acc_se*100:4.1f}%\t"
              f"{util_mean:+.3f} ± {util_se:.3f}")
        csv_prints.append(f"{unsure_mean*100:5.1f}% ± {unsure_se*100:4.1f}%, "
              f"{tot_acc*100:5.1f}% ± {tot_se*100:4.1f}%, "
              f"{acc_mean*100:5.1f}% ± {acc_se*100:4.1f}%, "
              f"{util_mean:+.3f} ± {util_se:.3f}")
        
    print(", ".join(csv_prints))

    # ---------------------------
    # Utility Gains Between Stages
    # ---------------------------
    gain_explanation = utilities['withexplanation'] - utilities['answeronly']
    gain_quality = utilities['withexplanationquality'] - utilities['withexplanation']
    expl_gain_mean = gain_explanation.mean()
    expl_gain_se = gain_explanation.std(ddof=1) / np.sqrt(len(gain_explanation)) if len(gain_explanation) > 1 else 0
    qual_gain_mean = gain_quality.mean()
    qual_gain_se = gain_quality.std(ddof=1) / np.sqrt(len(gain_quality)) if len(gain_quality) > 1 else 0
    
    print(f"\nSetting: {setting}")
    print("Stage\t\tUnsure Rate\tUtility over previous stage")
    print(f"Explanation\t {metrics['withexplanation']['unsure_rate_mean']*100:5.1f}%\t\t {expl_gain_mean:+.3f} ± {expl_gain_se:.3f}")
    print(f"Quality\t\t {metrics['withexplanationquality']['unsure_rate_mean']*100:5.1f}%\t\t {qual_gain_mean:+.3f} ± {qual_gain_se:.3f}")
    
    # ---------------------------
    # Compute Confusion Matrix and M1–M4 Scores
    # ---------------------------
    # Compute utility for withexplanation and withexplanationquality directly.
    def compute_utility(val, gt):
        return 0 if val == 2 else 1 - 2*abs(val-gt)
    
    df_valid['util_withexplanation'] = df_valid.apply(lambda row: compute_utility(row['withexplanation'], row['ground_truth']), axis=1)
    df_valid['util_withexplanationquality'] = df_valid.apply(lambda row: compute_utility(row['withexplanationquality'], row['ground_truth']), axis=1)
    
    # For the confusion matrix, map utilities to indices: -1 -> 0, 0 -> 1, 1 -> 2.
    mapping = {-1: 0, 0: 1, 1: 2}
    label_order = [-1, 0, 1]
    label_names = ['Incorrect', 'Unsure', 'Correct']
    
    try:
        cm = np.zeros((3, 3), dtype=int)
        for _, row in df_valid.iterrows():
            u2 = row['util_withexplanation']
            u3 = row['util_withexplanationquality']
            cm[mapping[u2], mapping[u3]] += 1
        cm_transposed = cm.T
    except Exception as e:
        print(f"Error computing confusion matrix or M1–M4 scores: {e}")
        print(f"This would be expected if setting {setting} is combined12 or combined123.")
        continue
        
    print(f"\nSetting: {setting} Step 2 -> Step 3 (transposed)")
    print("\t\t" + "\t".join(label_names))
    for i, label in enumerate(label_names):
        row_values = "\t".join(str(cm_transposed[i, j]) for j in range(3))
        print(f"{label:<10}\t {row_values}")
    
    # Compute M1–M4 using counts from the confusion matrix.
    def count_transitions(val2, val3):
        return ((df_valid['util_withexplanation'] == val2) & (df_valid['util_withexplanationquality'] == val3)).sum()
    
    denom_M1 = count_transitions(0, -1) + count_transitions(0, 0) + count_transitions(0, 1)
    num_M1 = count_transitions(0, -1) + count_transitions(0, 1)
    m1 = num_M1 / denom_M1 if denom_M1 > 0 else np.nan

    denom_M2 = count_transitions(0, -1) + count_transitions(0, 1)
    m2 = count_transitions(0, 1) / denom_M2 if denom_M2 > 0 else np.nan

    denom_M3 = count_transitions(-1, -1) + count_transitions(-1, 0) + count_transitions(-1, 1)
    num_M3 = count_transitions(-1, 0) + count_transitions(-1, 1)
    m3 = num_M3 / denom_M3 if denom_M3 > 0 else np.nan

    denom_M4 = count_transitions(1, -1) + count_transitions(1, 0) + count_transitions(1, 1)
    num_M4 = count_transitions(1, -1) + count_transitions(1, 0)
    m4 = num_M4 / denom_M4 if denom_M4 > 0 else np.nan

    print("\nM1–M4 Scores:")
    print(f"M1 (P(S3 ≠ U | S2 = U)): {m1:.2%}")
    print(f"M2 (P(S3 = C | S2 = U, S3 ≠ U)): {m2:.2%}")
    print(f"M3 (P(S3 ≠ I | S2 = I)): {m3:.2%}")
    print(f"M4 (P(S3 ≠ C | S2 = C)): {m4:.2%}")
    print(f"{m1:.2%}, {m2:.2%}, {m3:.2%}, {m4:.2%}, {m3-m4:.2%}")

    
    # ---------------------------
    # Print Examples Where Step 2 is Correct and Step 3 is Incorrect
    # ---------------------------
    print("\nExamples where Step 2 is correct and Step 3 is incorrect:")
    examples = df_valid[
        (df_valid['withexplanation'] != 2) &
        (df_valid['withexplanation'] == df_valid['ground_truth']) &
        (df_valid['withexplanationquality'] != 2) &
        (df_valid['withexplanationquality'] != df_valid['ground_truth'])
    ]
    if not examples.empty:
        for idx, row in examples.head(5).iterrows():
            print(row.to_json())
    else:
        print("No examples found.")
    
    # ---------------------------
    # Plot: Confusion Matrix and M1–M4 Scores for the Current Setting
    # ---------------------------
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    
    # Plot confusion matrix
    ax = axes[0]
    im = ax.imshow(cm_transposed, cmap='Greens', interpolation='nearest')
    ax.set_title(f"Setting {setting.split('_llava')[0]} Step 2->Step 3\n(WithExplanation -> WithExplanationQuality)")
    ax.set_xticks(np.arange(len(label_names)))
    ax.set_yticks(np.arange(len(label_names)))
    ax.set_xticklabels(label_names)
    ax.set_yticklabels(label_names)
    ax.set_xlabel("WithExplanation")
    ax.set_ylabel("WithExplanationQuality")
    for i in range(len(label_names)):
        for j in range(len(label_names)):
            ax.text(j, i, cm_transposed[i, j], ha="center", va="center", color="black")
    
    # Plot M1–M4 scores as text.
    ax = axes[1]
    ax.axis('off')
    score_text = (f"M1 (P(S3 ≠ U | S2 = U)): {m1:.2%}\n"
                  f"M2 (P(S3 = C | S2 = U, S3 ≠ U)): {m2:.2%}\n"
                  f"M3 (P(S3 ≠ I | S2 = I)): {m3:.2%}\n"
                  f"M4 (P(S3 ≠ C | S2 = C)): {m4:.2%}")
    ax.text(0.5, 0.5, score_text, fontsize=12, ha='center', va='center', transform=ax.transAxes)
    ax.set_title("M1–M4 Scores")
    plt.tight_layout()
    plt.show()
    
    # Save scores and confusion matrix for later comparison/plotting.
    all_setting_scores.append({
        "Setting": setting,
        "Resolution Rate": m1,
        "Positive Conversion Rate": m2,
        "Error Recovery Rate": m3,
        "Correct Loss Rate": m4
    })
    all_confusion_matrices.append((setting, cm_transposed))

# ---------------------------
# Plot Grouped Bar Chart for M1–M4 Scores Across Settings
# ---------------------------
if all_setting_scores:
    df_scores = pd.DataFrame(all_setting_scores)
    df_scores["SettingShort"] = df_scores["Setting"].apply(lambda s: s.split('_llava')[0])
    df_scores["Recovery Gap"] = df_scores["Error Recovery Rate"] - df_scores["Correct Loss Rate"]
    df_scores = df_scores.set_index("SettingShort")
    
    # Transpose so that rows become score types.
    df_scores_T = df_scores[["Resolution Rate", "Positive Conversion Rate", "Error Recovery Rate",
                             "Correct Loss Rate", "Recovery Gap"]].transpose()
    
    ax = df_scores_T.plot(kind='bar', figsize=(12, 6))
    ax.set_ylabel("Score (Fraction)")
    ax.set_title("Scores by Score Type Across Settings")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(title="Setting", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# ---------------------------
# Plot a Grid of Confusion Matrices for Every Setting
# ---------------------------
if all_confusion_matrices:
    n_settings = len(all_confusion_matrices)
    ncols = 2
    nrows = math.ceil(n_settings / ncols)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*5, nrows*5))
    if n_settings > 1:
        axes = np.array(axes).flatten()
    else:
        axes = [axes]
        
    for ax, (setting, cm_transposed) in zip(axes, all_confusion_matrices):
        im = ax.imshow(cm_transposed, cmap='Greens', interpolation='nearest')
        ax.set_title(setting.split('_llava')[0])
        ax.set_xticks(np.arange(len(label_names)))
        ax.set_yticks(np.arange(len(label_names)))
        ax.set_xticklabels(label_names)
        ax.set_yticklabels(label_names)
        ax.set_xlabel("WithExplanation")
        ax.set_ylabel("WithExplanationQuality")
        for i in range(len(label_names)):
            for j in range(len(label_names)):
                ax.text(j, i, cm_transposed[i, j], ha="center", va="center", color="black")
                
    # Turn off any extra axes.
    for ax in axes[len(all_confusion_matrices):]:
        ax.axis('off')
        
    plt.tight_layout()
    plt.show()



Processing setting: combined12_llava1.5_aokvqa_better_sampled_q10_i10_s0
                      user_id       batch  question_i  answeronly  \
0    66b6cb18cc43c649a5917319  000_3users           0          -1   
1    66b6cb18cc43c649a5917319  000_3users           1          -1   
2    66b6cb18cc43c649a5917319  000_3users           2          -1   
3    66b6cb18cc43c649a5917319  000_3users           3          -1   
4    66b6cb18cc43c649a5917319  000_3users           4          -1   
..                        ...         ...         ...         ...   
295  663abe859bcc3ea8ef9bbdf3  009_3users           5          -1   
296  663abe859bcc3ea8ef9bbdf3  009_3users           6          -1   
297  663abe859bcc3ea8ef9bbdf3  009_3users           7          -1   
298  663abe859bcc3ea8ef9bbdf3  009_3users           8          -1   
299  663abe859bcc3ea8ef9bbdf3  009_3users           9          -1   

     withexplanation  withexplanationquality  ground_truth  
0                 -1                

/tmp/ipykernel_16134/604509645.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_valid = df_valid.groupby('batch').apply(lambda g: g.head(30)).reset_index(drop=True)
/tmp/ipykernel_16134/604509645.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_valid = df_valid.groupby('batch').apply(lambda g: g.head(30)).reset_index(drop=True)
/tmp/ipykernel_16134/604509645.py:143: DeprecationWarning: DataF

30 users remain after filtering; 300 interactions remain.

Stage                    	Unsure Rate	TotalAcc	NotUnsureAcc	Utility
Answer Only              	  0.0% ±  0.0%	  0.0% ±  0.0%	  0.0% ±  0.0%	-2.000 ± 0.058
With Explanation         	  0.0% ±  0.0%	  0.0% ±  0.0%	  0.0% ±  0.0%	-2.000 ± 0.058
With Explanation + Quality	  7.3% ±  1.5%	 58.3% ±  2.9%	 62.9% ±  2.9%	+0.240 ± 0.054
  0.0% ±  0.0%,   0.0% ±  0.0%,   0.0% ±  0.0%, -2.000 ± 0.058,   0.0% ±  0.0%,   0.0% ±  0.0%,   0.0% ±  0.0%, -2.000 ± 0.058,   7.3% ±  1.5%,  58.3% ±  2.9%,  62.9% ±  2.9%, +0.240 ± 0.054

Setting: vf_numeric_llava1.5_aokvqa_better_sampled_q10_i10_s0
Stage		Unsure Rate	Utility over previous stage
Explanation	   0.0%		 +0.000 ± 0.000
Quality		   7.3%		 +2.240 ± 0.055
Error computing confusion matrix or M1–M4 scores: -3
This would be expected if setting vf_numeric_llava1.5_aokvqa_better_sampled_q10_i10_s0 is combined12 or combined123.

Processing setting: vf_descriptive_llava1.5_aokvqa_better_sampled_q10_i

/tmp/ipykernel_16134/604509645.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_valid = df_valid.groupby('batch').apply(lambda g: g.head(30)).reset_index(drop=True)


In [5]:
# Store df_map to a file for later use.
import pickle
for setting, df in df_map.items():
    # only keep the columns we need
    df_map[setting] = df[['user_id', 'batch', 'question_i', 'withexplanationquality', 'ground_truth']]
with open('df_combined123_map.pkl', 'wb') as f:
    pickle.dump(df_map, f)

# Start from here for the analysis of user interaction data vs. confidence score

In [ ]:
import pickle
# Load df_map from the file.
with open('df_map.pkl', 'rb') as f:
    df_map = pickle.load(f)
print("Loaded df_map from file.")
# print the keys of df_map
print("Keys in df_map:")
for key in df_map.keys():
    print(key)
df_map['vf_numeric_llava1.5_with_image_q20_i10_s0']

In [8]:
import pickle
# Load df_combined123_map from the file.
with open('df_combined123_map.pkl', 'rb') as f:
    df_combined123_map = pickle.load(f)
print("Loaded df_combined123_map from file.")
# print the keys of df_combined123_map
print("Keys in df_combined123_map:")
for key in df_combined123_map.keys():
    print(key)
df_combined123_map['vf_numeric_llava1.5_aokvqa_better_sampled_q10_i10_s0']

Loaded df_combined123_map from file.
Keys in df_combined123_map:
combined12_llava1.5_aokvqa_better_sampled_q10_i10_s0
prodmetric_llava1.5_aokvqa_better_sampled_q10_i10_s0
vf_numeric_llava1.5_aokvqa_better_sampled_q10_i10_s0
vf_descriptive_llava1.5_aokvqa_better_sampled_q10_i10_s0


,user_id,batch,question_i,withexplanationquality,ground_truth
0,63483c4522d65d46028da7a7,000_3users,0,0,0
1,63483c4522d65d46028da7a7,000_3users,1,0,0
2,63483c4522d65d46028da7a7,000_3users,2,0,0
3,63483c4522d65d46028da7a7,000_3users,3,0,0
4,63483c4522d65d46028da7a7,000_3users,4,0,1
...,...,...,...,...,...
295,672a6ed8d37a4143e096929b,009_3users,5,0,1
296,672a6ed8d37a4143e096929b,009_3users,6,0,0
297,672a6ed8d37a4143e096929b,009_3users,7,0,0
298,672a6ed8d37a4143e096929b,009_3users,8,0,0


In [ ]:
import pickle
import pandas as pd
import numpy as np

# 1) Some helper functions
def calc_util(row):
    p  = row['withexplanationquality']    # 0=trust, 1=distrust, 2=unsure
    gt = row['ground_truth']              # 0=correct, 1=incorrect
    if p == 2:
        return 0
    # for p in {0,1}, utility = +1 if (p==gt) else –1
    return 1 - 2 * abs(p - gt)

# 2) For each model‐variant DF, compute metrics
rows = []
for key, df in df_combined123_map.items():
    n = len(df)
    p  = df['withexplanationquality']
    gt = df['ground_truth']
    
    unsure_count  = (p == 2).sum()
    accept_count  = (p == 0).sum()
    
    # “Correct predictions” = user trusts correct OR user rejects incorrect
    correct_preds = ((p == 0) & (gt == 0) | (p == 1) & (gt == 1)).sum()
    
    # Rates
    unsure_rate     = unsure_count / n
    accept_rate     = accept_count / n
    total_acc       = correct_preds / n
    not_unsure_acc  = correct_preds / (n - unsure_count) if (n - unsure_count)>0 else np.nan
    false_acc_rate  = ((p == 0) & (gt == 1)).sum() / (gt == 1).sum()
    false_rej_rate  = ((p == 1) & (gt == 0)).sum() / (gt == 0).sum()
    
    # Utility per row -> mean
    df['utility']   = df.apply(calc_util, axis=1)
    mean_util       = df['utility'].mean()
    
    rows.append({
        'model_variant': key,
        'Unsure Rate':    unsure_rate,
        'Accept Rate':    accept_rate,
        'TotalAcc':       total_acc,
        'NotUnsureAcc':   not_unsure_acc,
        'False Acc Rate': false_acc_rate,
        'False Rej Rate': false_rej_rate,
        'Mean Utility':   mean_util
    })

# 3) Summarize
summary_df = pd.DataFrame(rows).set_index('model_variant')
display(summary_df)


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# --- Settings to compare ---
settings = [
    'vf_numeric_llava1.5_with_image_q20_i10_s0',
    'contr_numeric_llava1.5_with_image_q20_i10_s0',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0',
    'vf_descriptive_llava1.5_with_image_q20_i10_s0',
    'contr_descriptive_llava1.5_with_image_q20_i10_s0',
    'vf_numeric_qwen2.5_vizwiz_q10_i10_s0',
    'vf_descriptive_qwen2.5_vizwiz_q10_i10_s0',
]

records = []
for setting in settings:
    df = df_map[setting]
    for idx, row in df.iterrows():
        queue_name = "qwen2.5_vizwiz_q10_i10_s0" if 'vizwiz' in setting else "llava1.5_with_image_q20_i10_s0"
        # get the batch ID (e.g. '000' from '000_3users')
        batch_id = row['batch'].split('_')[0]
        path = f'../web/baked_queues/{queue_name}/{batch_id}.json'
        if not os.path.exists(path):
            continue

        # load the questions for this batch
        with open(path) as f:
            questions = json.load(f)
        entry = questions[row['question_i']]

        vf   = entry.get('visual_fidelity')
        ctr  = entry.get('contrastiveness')
        avg  = None if vf is None or ctr is None else (vf + ctr) / 2

        # pick which score was shown
        if 'avg_vf_contr' in setting:
            confidence = avg
        elif setting.startswith('vf'):
            confidence = vf
        elif setting.startswith('contr'):
            confidence = ctr
        else:
            condidence = None
        # add to df
        df.loc[idx, 'confidence'] = confidence
    
df_map['vf_numeric_llava1.5_with_image_q20_i10_s0']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# mapping from your raw column names to nice labels
stage_map = {
    'answeronly': 'Answer Only',
    'withexplanation': 'With Explanation',
    'withexplanationquality': 'With Explanation + Quality'
}

selection_order = [0, 1, 2]
colors = ['tab:green', 'tab:red', 'tab:orange']
stage_order = list(stage_map.values())

for setting, df in df_map.items():
    # 1) melt into long form
    df_long = df.melt(
        id_vars=['user_id', 'batch', 'question_i', 'ground_truth'],
        value_vars=list(stage_map.keys()),
        var_name='Stage',
        value_name='Selection'
    )
    # 2) map raw to pretty
    df_long['Stage'] = df_long['Stage'].map(stage_map)

    # 3) group + count
    grouped = (
        df_long
        .groupby('Stage')['Selection']
        .value_counts()
        .unstack(fill_value=0)
        .reindex(index=stage_order, columns=selection_order)
    )

    # 4) plot stacked bar
    fig, ax = plt.subplots(figsize=(6, 8))
    x = np.arange(len(stage_order))
    bottom = np.zeros(len(stage_order))

    for sel, color in zip(selection_order, colors):
        counts = grouped[sel].values
        legend_label = f"Correct" if sel == 0 else f"Incorrect" if sel == 1 else "Unsure"
        bars = ax.bar(x, counts, bottom=bottom,
                      label=legend_label,
                      color=color, edgecolor='black')
        # annotate counts
        for idx, bar in enumerate(bars):
            h = bar.get_height()
            if h:
                ax.text(
                    bar.get_x() + bar.get_width()/2,
                    bottom[idx] + h/2,
                    f"{int(h)}",
                    ha='center', va='center'
                )
        bottom += counts

    ax.set_xticks(x)
    ax.set_xticklabels(stage_order, fontsize=12, rotation=30)
    ax.set_xlabel("Stage", fontsize=14)
    ax.set_ylabel("Count", fontsize=14)
    ax.set_title(f"Selection Counts by Stage ({setting.split('_llava')[0].split('_vizwiz')[0]})",
                 fontsize=16)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# mapping from your raw column names to nice labels
stage_map = {
    'answeronly': 'Answer Only',
    'withexplanation': 'With Explanation',
    'withexplanationquality': 'With Explanation + Quality'
}

selection_order = [0, 1, 2]
colors = ['tab:green', 'tab:red', 'tab:orange']
stage_order = list(stage_map.values())

for setting, df in df_map.items():
    # 1) melt into long form
    df_long = df.melt(
        id_vars=['user_id', 'batch', 'question_i', 'ground_truth'],
        value_vars=list(stage_map.keys()),
        var_name='Stage',
        value_name='Selection'
    )
    # 2) map raw to pretty
    df_long['Stage'] = df_long['Stage'].map(stage_map)

    # 3) compute u_i
    #   u_i = 0 if Selection == 2
    #   u_i = 1 if (sel=1 & gt=0) or (sel=0 & gt=1)
    #   u_i = -1 otherwise (i.e. sel matches gt)
    df_long['u_i'] = np.where(
        df_long['Selection'] == 2, 0,
        np.where(
            ((df_long['Selection'] == 1) & (df_long['ground_truth'] == 1)) |
            ((df_long['Selection'] == 0) & (df_long['ground_truth'] == 0)),
            1,
            -1
        )
    )

    # inspect the new column
    print(f"\n—— {setting} ——")
    # randomly sample 5 rows
    print(df_long[['Stage', 'Selection', 'ground_truth', 'u_i']].sample(5))

    # now group & plot u_i
    u_order  = [1, -1, 0]                     # mismatch, unsure, match
    u_colors = ['tab:green', 'tab:red', 'tab:orange',]
    u_labels = ['Match (+1)', 'Mismatch (-1)', 'Unsure (0)',]

    grouped = (
        df_long
        .groupby('Stage')['u_i']
        .value_counts()
        .unstack(fill_value=0)
        .reindex(index=stage_order, columns=u_order)
    )

    fig, ax = plt.subplots(figsize=(6, 8))
    x = np.arange(len(stage_order))
    bottom = np.zeros(len(stage_order))

    for u, color, label in zip(u_order, u_colors, u_labels):
        counts = grouped[u].values
        bars = ax.bar(x, counts, bottom=bottom, label=label,
                    color=color, edgecolor='black')
        for idx, bar in enumerate(bars):
            h = bar.get_height()
            if h:
                ax.text(bar.get_x() + bar.get_width()/2,
                        bottom[idx] + h/2,
                        f"{int(h)}",
                        ha='center', va='center')
        bottom += counts

    ax.set_xticks(x)
    ax.set_xticklabels(stage_order, fontsize=12, rotation=30)
    ax.set_xlabel("Stage", fontsize=14)
    ax.set_ylabel("Count", fontsize=14)
    ax.set_title(f"Answer Correctness by Stage ({setting.split('_llava')[0].split('_vizwiz')[0]})",
                fontsize=16)
    ax.legend(fontsize=12, title="Correctness", loc='upper left')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import PercentFormatter

# --- your existing mappings & lists ---
stage_map = {
    'answeronly': 'Answer Only',
    'withexplanation': 'With Explanation',
    'withexplanationquality': 'With Explanation + Quality'
}
selection_order = [0, 1, 2]
colors = ['tab:green', 'tab:red', 'tab:orange']
labels = {
    0: 'Correct',
    1: 'Incorrect',
    2: 'Unsure'
}
applied_settings = [
    'vf_numeric_llava1.5_with_image_q20_i10_s0',
    'contr_numeric_llava1.5_with_image_q20_i10_s0',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0',
    'showbothmetrics_llava1.5_with_image_q20_i10_s0',
]
applied_setting_names = {
    'vf_numeric_llava1.5_with_image_q20_i10_s0': 'VF Numeric',
    'contr_numeric_llava1.5_with_image_q20_i10_s0': 'CONTR Numeric',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0': 'Average VF/CONTR',
    'showbothmetrics_llava1.5_with_image_q20_i10_s0': 'Both Metrics',
}

# 1) Melt & compute u_i for all settings (same as before) …
all_dfs = []
for setting, df in df_map.items():
    if setting not in applied_settings:
        continue
    df_long = df.melt(
        id_vars=['user_id','batch','question_i','ground_truth'],
        value_vars=list(stage_map.keys()),
        var_name='Stage', value_name='Selection'
    )
    df_long['Stage'] = df_long['Stage'].map(stage_map)
    df_long['Setting'] = setting

    all_dfs.append(df_long)
big = pd.concat(all_dfs, ignore_index=True)

# 2a) Overall Stage 2 (“With Explanation”)
agg2 = (big.loc[big['Stage']=='With Explanation','Selection']
          .value_counts()
          .reindex(selection_order, fill_value=0))
agg2_prop = agg2 / agg2.sum()

# 2b) Stage 3 (“With Explanation + Quality”) per setting
stage3 = big[big['Stage']=='With Explanation + Quality']
by_set = (stage3.groupby('Setting')['Selection']
               .value_counts()
               .unstack(fill_value=0)
               .reindex(columns=selection_order, fill_value=0))
by_set_prop = by_set.div(by_set.sum(axis=1), axis=0)

# 3) Build combined_prop in the exact order: Overall, then each applied_setting
rows = [agg2_prop] + [by_set_prop.loc[s] for s in applied_settings]
index = ['Overall'] + applied_settings
combined_prop = pd.DataFrame(rows, index=index, columns=selection_order)

# 4) Plot them with tighter bars
fig, ax = plt.subplots(figsize=(8,6))
x = np.arange(len(combined_prop))
width = 0.8   
bottom = np.zeros(len(combined_prop))

for u, color in zip(selection_order, colors):
    vals = combined_prop[u]
    bars = ax.bar(x, vals, width, bottom=bottom,
                  color=color, label=labels[u], edgecolor='k')
    for xi, (bar, v) in enumerate(zip(bars, vals)):
        if v > 0:
            ax.text(
                bar.get_x() + bar.get_width()/2,
                bottom[xi] + v/2,
                f"{v:.1%}",
                ha='center', va='center'
            )
    bottom += vals

# 5) Force x‑labels in the same exact order
ax.set_xticks(x)
ax.set_xticklabels(
    ['With Explanation'] +
    [applied_setting_names[s] for s in applied_settings],
    rotation=30, ha='right'
)

ax.margins(x=0.02)
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.set_ylabel('Percentage', fontsize=14)
ax.set_title('User Selection Across Settings', fontsize=16)
ax.legend(title='Selection', loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import PercentFormatter

# --- your existing mappings & lists ---
stage_map = {
    'answeronly': 'Answer Only',
    'withexplanation': 'With Explanation',
    'withexplanationquality': 'With Explanation + Quality'
}
selection_order = [1, -1, 0]
colors = ['tab:green', 'tab:red', 'tab:orange']
labels = {
    1: 'Match (+1)',
   -1: 'Mismatch (-1)',
    0: 'Unsure (0)'
}
applied_settings = [
    'vf_numeric_llava1.5_with_image_q20_i10_s0',
    'contr_numeric_llava1.5_with_image_q20_i10_s0',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0',
    'showbothmetrics_llava1.5_with_image_q20_i10_s0',
]
applied_setting_names = {
    'vf_numeric_llava1.5_with_image_q20_i10_s0': 'VF Numeric',
    'contr_numeric_llava1.5_with_image_q20_i10_s0': 'CONTR Numeric',
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0': 'Average VF/CONTR',
    'showbothmetrics_llava1.5_with_image_q20_i10_s0': 'Both Metrics',
}

# 1) Melt & compute u_i for all settings (same as before) …
all_dfs = []
for setting, df in df_map.items():
    if setting not in applied_settings:
        continue
    df_long = df.melt(
        id_vars=['user_id','batch','question_i','ground_truth'],
        value_vars=list(stage_map.keys()),
        var_name='Stage', value_name='Selection'
    )
    df_long['Stage'] = df_long['Stage'].map(stage_map)
    df_long['Setting'] = setting
    df_long['u_i'] = np.where(
        df_long['Selection']==2, 0,
        np.where(
            ((df_long['Selection']==1)&(df_long['ground_truth']==1)) |
            ((df_long['Selection']==0)&(df_long['ground_truth']==0)),
            1,
            -1
        )
    )
    all_dfs.append(df_long)
big = pd.concat(all_dfs, ignore_index=True)

# 2a) Overall Stage 2 (“With Explanation”)
agg2 = (big.loc[big['Stage']=='With Explanation','u_i']
          .value_counts()
          .reindex(selection_order, fill_value=0))
agg2_prop = agg2 / agg2.sum()

# 2b) Stage 3 (“With Explanation + Quality”) per setting
stage3 = big[big['Stage']=='With Explanation + Quality']
by_set = (stage3.groupby('Setting')['u_i']
               .value_counts()
               .unstack(fill_value=0)
               .reindex(columns=selection_order, fill_value=0))
by_set_prop = by_set.div(by_set.sum(axis=1), axis=0)

# 3) Build combined_prop in the exact order: Overall, then each applied_setting
rows = [agg2_prop] + [by_set_prop.loc[s] for s in applied_settings]
index = ['Overall'] + applied_settings
combined_prop = pd.DataFrame(rows, index=index, columns=selection_order)

# 4) Plot them with tighter bars
fig, ax = plt.subplots(figsize=(8,6))
x = np.arange(len(combined_prop))
width = 0.8   
bottom = np.zeros(len(combined_prop))

for u, color in zip(selection_order, colors):
    vals = combined_prop[u]
    bars = ax.bar(x, vals, width, bottom=bottom,
                  color=color, label=labels[u], edgecolor='k')
    for xi, (bar, v) in enumerate(zip(bars, vals)):
        if v > 0:
            ax.text(
                bar.get_x() + bar.get_width()/2,
                bottom[xi] + v/2,
                f"{v:.1%}",
                ha='center', va='center'
            )
    bottom += vals

# 5) Force x‑labels in the same exact order
ax.set_xticks(x)
ax.set_xticklabels(
    ['With Explanation'] +
    [applied_setting_names[s] for s in applied_settings],
    rotation=30, ha='right'
)

ax.margins(x=0.02)
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.set_ylabel('Percentage', fontsize=14)
ax.set_title('Answer Correctness by Setting (in %)', fontsize=16)
ax.legend(title='Correctness', loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
for setting, df in df_map.items():
    df['utility_2'] = np.where(
        df['withexplanation'] == 2, 0,
        np.where(
            ((df['withexplanation'] == 1) & (df['ground_truth'] == 1)) |
            ((df['withexplanation'] == 0) & (df['ground_truth'] == 0)),
            1,
            -1
        )
    )
    df['utility_3'] = np.where(
        df['withexplanationquality'] == 2, 0,
        np.where(
            ((df['withexplanationquality'] == 1) & (df['ground_truth'] == 1)) |
            ((df['withexplanationquality'] == 0) & (df['ground_truth'] == 0)),
            1,
            -1
        )
    )
df_map['vf_numeric_llava1.5_with_image_q20_i10_s0'].sample(10)


In [ ]:
# 2) Compute M2–M4 using utility, for each applied setting
all_scores = []
for setting in applied_settings:
    df = df_map[setting]
    
    # M2: Positive conversion rate
    #    P(u3 = +1 | u2 = 0, u3 != 0)
    mask2 = (df['utility_2'] == 0) & (df['utility_3'] != 0)
    m2 = (df.loc[mask2, 'utility_3'] == 1).sum() / mask2.sum()
    
    # M3: Error recovery rate
    #    P(u3 >= 0 | u2 < 0)
    mask3 = df['utility_2'] < 0
    m3 = (df.loc[mask3, 'utility_3'] >= 0).sum() / mask3.sum()
    
    # M4: Correct loss rate
    #    P(u3 <= 0 | u2 > 0)
    mask4 = df['utility_2'] > 0
    m4 = (df.loc[mask4, 'utility_3'] <= 0).sum() / mask4.sum()
    
    all_scores.append({
        'Setting': applied_setting_names[setting],
        'Positive Conversion': m2,
        'Error Recovery': m3,
        'Correct Loss': m4
    })

# 3) Build DataFrame and plot grouped bar chart
df_scores = pd.DataFrame(all_scores).set_index('Setting')

ax = df_scores.plot(
    kind='bar',
    figsize=(10, 6),
    width=0.6,
    edgecolor='k'
)
ax.set_xlabel('Setting', fontsize=16)
ax.set_ylabel('Rate', fontsize=18)
ax.set_ylim(0, 1)
ax.set_title('Correctness Change Measures Across Settings', fontsize=16)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=16)
ax.legend(title='Metric', loc='upper right', fontsize=16, title_fontsize=16)
plt.tight_layout()
plt.show()

# 4) Group by metric on the x‑axis instead of by setting
#    i.e. transpose so rows become metrics and columns become settings
df_scores_T = df_scores.transpose()

ax = df_scores_T.plot(
    kind='bar',
    figsize=(10, 6),
    width=0.6,
    edgecolor='k'
)
ax.set_xlabel('Metric', fontsize=16)
ax.set_ylabel('Rate', fontsize=18)
ax.set_title('Correctness Change Measures', fontsize=16)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=16)
ax.legend(title='Setting', loc='upper right', fontsize=16, title_fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# 5) Plot M3 (Error Recovery) for VF Numeric vs VF Descriptive
# -------------------------------------------------------------

numeric_descriptive_settings_pair = [
    ['vf_numeric_llava1.5_with_image_q20_i10_s0', 'vf_descriptive_llava1.5_with_image_q20_i10_s0'],
    ['contr_numeric_llava1.5_with_image_q20_i10_s0', 'contr_descriptive_llava1.5_with_image_q20_i10_s0'],
    ['showbothmetrics_llava1.5_with_image_q20_i10_s0', 'showbothmetrics_descriptive_llava1.5_with_image_q20_i10_s0'],
]

# Define group labels
group_labels = ['VF', 'CONTR', 'Both']

# Compute M3 for each group
m3_dict = {}
for label, (num_s, des_s) in zip(group_labels, numeric_descriptive_settings_pair):
    df_num = df_map[num_s]
    df_des = df_map[des_s]
    
    mask_num = df_num['utility_2'] < 0
    mask_des = df_des['utility_2'] < 0
    
    m3_num = (df_num.loc[mask_num, 'utility_3'] >= 0).sum() / mask_num.sum()
    m3_des = (df_des.loc[mask_des, 'utility_3'] >= 0).sum() / mask_des.sum()
    
    m3_dict[label] = {'Numeric': m3_num, 'Descriptive': m3_des}

# Build DataFrame
m3_df = pd.DataFrame.from_dict(m3_dict, orient='index')

# Plot grouped bars
fig, ax = plt.subplots(figsize=(8, 5))
m3_df.plot(
    kind='bar',
    ax=ax,
    width=0.6,
    edgecolor='k'
)

# Tighten spacing
ax.margins(x=0.02)

# Labels and title
ax.set_ylabel('Error Recovery Rate', fontsize=16)
# ax.set_ylim(0, 1)
ax.set_title('Error Recovery Rate by Numeric vs Descriptive', fontsize=14)
ax.set_xticklabels(group_labels, rotation=0, fontsize=16)

# Annotate bar values
for i, row_label in enumerate(group_labels):
    for j, col_label in enumerate(['Numeric', 'Descriptive']):
        v = m3_df.loc[row_label, col_label]
        ax.text(i + (j-0.5)*0.3, v, f"{v:.1%}",
                ha='center', va='bottom', fontsize=10)

# Legend
ax.legend(title='Format', loc='upper left')

plt.tight_layout()
plt.show()

# Compute M4 for each group
m4_dict = {}
for label, (num_s, des_s) in zip(group_labels, numeric_descriptive_settings_pair):
    df_num = df_map[num_s]
    df_des = df_map[des_s]
    
    mask_num = df_num['utility_2'] > 0
    mask_des = df_des['utility_2'] > 0
    
    m4_num = (df_num.loc[mask_num, 'utility_3'] <= 0).sum() / mask_num.sum()
    m4_des = (df_des.loc[mask_des, 'utility_3'] <= 0).sum() / mask_des.sum()
    
    m4_dict[label] = {'Numeric': m4_num, 'Descriptive': m4_des}
# Build DataFrame
m4_df = pd.DataFrame.from_dict(m4_dict, orient='index')
# Plot grouped bars
fig, ax = plt.subplots(figsize=(8, 5))
m4_df.plot(
    kind='bar',
    ax=ax,
    width=0.6,
    edgecolor='k'
)
# Tighten spacing
ax.margins(x=0.02)
# Labels and title
ax.set_ylabel('Correct Loss Rate', fontsize=16)
# ax.set_ylim(0, 1)
ax.set_title('Correct Loss Rate by Numeric vs Descriptive', fontsize=14)
ax.set_xticklabels(group_labels, rotation=0, fontsize=16)
# Annotate bar values
for i, row_label in enumerate(group_labels):
    for j, col_label in enumerate(['Numeric', 'Descriptive']):
        v = m4_df.loc[row_label, col_label]
        ax.text(i + (j-0.5)*0.3, v, f"{v:.1%}",
                ha='center', va='bottom', fontsize=10)
# Legend
ax.legend(title='Format', loc='upper left')
plt.tight_layout()
plt.show()
# Compute M5 (Recovery Gap) correctly for each group
m5_dict = {}
for label, (num_s, des_s) in zip(group_labels, numeric_descriptive_settings_pair):
    df_num = df_map[num_s]
    df_des = df_map[des_s]
    
    # Mask for error cases (u2 < 0) and for correct cases (u2 > 0)
    mask_num_neg = df_num['utility_2'] < 0
    mask_num_pos = df_num['utility_2'] > 0
    mask_des_neg = df_des['utility_2'] < 0
    mask_des_pos = df_des['utility_2'] > 0
    
    # M3_num on negative mask, M4_num on positive mask
    m3_num = (df_num.loc[mask_num_neg, 'utility_3'] >= 0).sum()   / mask_num_neg.sum()
    m4_num = (df_num.loc[mask_num_pos, 'utility_3'] <= 0).sum()   / mask_num_pos.sum()
    
    # same for descriptive
    m3_des = (df_des.loc[mask_des_neg, 'utility_3'] >= 0).sum()   / mask_des_neg.sum()
    m4_des = (df_des.loc[mask_des_pos, 'utility_3'] <= 0).sum()   / mask_des_pos.sum()
    
    m5_dict[label] = {
        'Numeric': m3_num - m4_num,
        'Descriptive': m3_des - m4_des
    }

# Build and plot as before
m5_df = pd.DataFrame.from_dict(m5_dict, orient='index')
ax = m5_df.plot(kind='bar', figsize=(8,5), width=0.6, edgecolor='k')
ax.margins(x=0.02)
ax.set_ylabel('Recovery Gap', fontsize=16)
ax.set_title('Recovery Gap by Numeric vs Descriptive', fontsize=14)
ax.set_xticklabels(group_labels, rotation=0, fontsize=16)
for i, row_label in enumerate(group_labels):
    for j, col_label in enumerate(['Numeric','Descriptive']):
        v = m5_df.loc[row_label, col_label]
        ax.text(i + (j-0.5)*0.3, v, f"{v:.1%}", ha='center', va='bottom', fontsize=10)
ax.legend(title='Format', loc='upper left')
plt.tight_layout()
plt.show()




In [ ]:
# Plot the confidence distributions for each setting
for setting in settings:
    df = df_map[setting]
    plt.figure(figsize=(5, 2))
    plt.hist(df['confidence'], bins=20, alpha=0.7, label=setting)
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.title(f'Confidence Distribution for {setting}')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import statsmodels.api as sm

def bin_stats(df, not_wanted_value):
    n = len(df)
    if n == 0:
        return pd.Series({'p': 0, 'se': 0, 'n': 0})
    k = (df['withexplanationquality'] != not_wanted_value).sum()
    p = k / n
    # Laplace-smoothed se
    p_ = (k + 1) / (n + 2)
    se_ = sqrt(p_ * (1 - p_) / (n + 2))
    return pd.Series({'p': p, 'se': se_, 'n': n})

# colors for each selection
color_map = {0: 'C0', 1: 'tab:green', 2: 'tab:orange'}

for setting, df in df_map.items():
    if setting not in settings:
        continue

    # choose metric label
    if 'avg_vf_contr' in setting:
        confidence_metric = 'AVERAGE(VF, CONTR)'
    elif setting.startswith('vf'):
        confidence_metric = 'VF'
    elif setting.startswith('contr'):
        confidence_metric = 'CONTR'
    else:
        confidence_metric = ''

    # bin edges
    n_bins = 5
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    width = (bin_edges[1] - bin_edges[0]) * 0.9
    
    df['conf_bin'] = pd.cut(
        df['confidence'],
        bins=bin_edges,
        include_lowest=True,  # makes 0.0 go into the first bin
        right=True            # makes 1.0 go into the last bin
    )

    # make 3 subplots
    fig, axes = plt.subplots(
        1, 3,
        figsize=(12, 4),
        sharey=True
    )

    for ax, sel in zip(axes, [0, 1, 2]):
        df0 = df[df['withexplanation'] == sel]
        grouped = (
            df0
            .groupby('conf_bin', observed=False)
            .apply(lambda g: bin_stats(g, sel))
            .reset_index()
        )
        # to percentages
        grouped['p']  *= 100
        grouped['se'] *= 100

        ax.bar(
            bin_centers,
            grouped['p'],
            width=width,
            yerr=grouped['se'],
            capsize=5,
            edgecolor='black',
            color=color_map[sel]
        )
        # annotate counts on top of each bar
        for x, height, count in zip(bin_centers, grouped['p'], grouped['n']):
            ax.text(
                x, 
                height + 1,              # small vertical offset
                f'n={int(count)}',
                ha='center', 
                va='bottom',
                fontsize='small'
            )
                
        # ---------------------------------------------------------------
        # Weighted Linear Regression using WLS (weights = counts, n)
        # ---------------------------------------------------------------
        x_data = bin_centers
        y_data = grouped['p']
        weights = grouped['n']
        
        # Remove potential bins with zero counts, which could be problematic
        valid = weights > 0
        x_valid = x_data[valid]
        y_valid = y_data[valid]
        weights_valid = weights[valid]
        
        # Only regress if we have 2+ points
        if len(x_valid) > 1:
            X = sm.add_constant(x_valid)
            wls_model = sm.WLS(y_valid, X, weights=weights_valid)
            results = wls_model.fit()
            intercept, slope = results.params
            slope_se = results.bse[1]  # standard error of slope
            # Create line for plotting
            x_line = np.linspace(min(x_valid), max(x_valid), 50)
            y_line = slope * x_line + intercept
            ax.plot(
                x_line, y_line,
                color='red', linestyle='--',
                label=(f"y = {slope:.2f}x + {intercept:.2f}\n"
                    f"Slope SE = {slope_se:.2f}")
            )

            ax.legend(fontsize='small')

        # ---------------------------------------------------------------
        

        ax.set_title(f's₂ == {sel}')
        ax.set_xlabel(f'Confidence ({confidence_metric})')
        ax.set_xlim(0, 1)


        
    axes[0].set_ylabel(f'P(s₃ ≠ s₂ ∣ s₂ == {{0,1,2}}) [%]')
    axes[0].set_ylim(0, 100)
    plt.suptitle(f"Setting: {setting}")
    plt.tight_layout(rect=[0, 0, 1, 0.90])
    plt.show()


In [ ]:
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

def bin_stats(df, not_wanted_value):
    n = len(df)
    if n == 0:
        return pd.Series({'p': 0, 'se': 0, 'n': 0})
    k = (df['withexplanationquality'] != not_wanted_value).sum()
    p = k / n
    # Laplace-smoothed standard error
    p_ = (k + 1) / (n + 2)
    se_ = sqrt(p_ * (1 - p_) / (n + 2))
    return pd.Series({'p': p, 'se': se_, 'n': n})

metrics_to_plot = ['vf_numeric_llava1.5_with_image_q20_i10_s0', 'vf_descriptive_llava1.5_with_image_q20_i10_s0']
metric_colors = {'VF_numeric': 'C0', 'VF_descriptive': 'C1'}
# a simple mapping from your metric‐IDs to pretty legend labels
label_map = {
    'vf_numeric_llava1.5_with_image_q20_i10_s0': 'VF numeric',
    'vf_descriptive_llava1.5_with_image_q20_i10_s0': 'VF descriptive',
}

n_bins = 5
bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
# Loop over the s₂ values – here using withexplanation == 0, 1, 2
for ax, sel in zip(axes, [0, 1, 2]):
    # For each s₂ subplot, plot both metrics on the same axes
    for metric in metrics_to_plot:
        # Get the corresponding DataFrame from df_map.
        # (For safety, we use a copy since we will add a new column.)
        df = df_map[metric].copy()
        # Bin the confidence values (you can adjust include_lowest/right as appropriate)
        df['conf_bin'] = pd.cut(
            df['confidence'],
            bins=bin_edges,
            include_lowest=True,
            right=True
        )
        
        # Select the data for the current s₂ value
        df0 = df[df['withexplanation'] == sel]
        
        # Group by confidence bin and calculate proportion (p), laplace-smoothed se, and count (n)
        grouped = (df0.groupby('conf_bin', observed=False)
                    .apply(lambda g: bin_stats(g, sel))
                    .reset_index())
        # (Optional) make sure that the results are ordered by bin
        grouped = grouped.sort_values(by='conf_bin')
        
        # Convert proportions to percentages
        grouped['p'] *= 100
        grouped['se'] *= 100
        
        # Plot a line with markers and error bars using errorbar()
        ax.errorbar(
            bin_centers,
            grouped['p'],
            yerr=grouped['se'],
            marker='o',
            linestyle='-',
            color=metric_colors.get(metric, None),
            capsize=5,
            label=label_map[metric]
        )
        
        # annotate counts on top of each bar
        height_adjust = -1 if metric == 'vf_numeric_llava1.5_with_image_q20_i10_s0' else 4
        for x, height, count in zip(bin_centers, grouped['p'], grouped['n']):
            ax.text(
                x + 0.08,              # small horizontal offset
                height + height_adjust if height < 99 else height - 10 + height_adjust,              # small vertical offset
                f'n={int(count)}',
                ha='center',
                va='bottom',
                fontsize='small',
                color=metric_colors['VF_numeric'] if 'numeric' in metric else metric_colors['VF_descriptive']
            )
        

    title = f"s{sel}: "
    if sel == 0:
        title += "User thinks AI is Correct"
    elif sel == 1:
        title += "User thinks AI is Incorrect"
    elif sel == 2:
        title += "User is Unsure"
    ax.set_title(f'{title}')
    ax.set_xlabel('Confidence')
    ax.set_xlim(0, 1)
    # Set legend name
    # ax.legend(fontsize='small')
    
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, ncol=3, fontsize='small')
axes[0].set_ylabel('Switch Rate at Stage 3 [%]', fontsize=14)
axes[0].set_ylim(0, 100)
plt.suptitle("After seeing explanation...", y=0.96)
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

In [ ]:
# Additional color mapping for compared metrics
compared_metric_colors = {
'vf_numeric_llava1.5_with_image_q20_i10_s0': 'C0',
'contr_numeric_llava1.5_with_image_q20_i10_s0': 'C2',
'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0': 'C3'
}

compared_metrics2 = ['vf_numeric_llava1.5_with_image_q20_i10_s0', 'contr_numeric_llava1.5_with_image_q20_i10_s0', 'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0']


# Update label_map if needed
label_map.update({
'contr_numeric_llava1.5_with_image_q20_i10_s0': 'Contr. numeric',
'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0': 'Avg VF Contr. numeric'
})

fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)

LABEL_PAD   = 2      # vertical step (points) you’re willing to move the label
MIN_SEP_Y   = 3      # how close two labels can be before we call it a collision
MIN_SEP_X   = 0.15   # so labels on different bars at same x don’t count
placed_xy   = []     # keeps track of every label we’ve already drawn

def find_free_y(x, y0):
    """
    Return the nearest y‑coordinate to y0 that is not 'too close' to any
    existing label.  We look up, then down, in LABEL_PAD‑point steps.
    """
    up_y   = y0
    down_y = y0
    # keep expanding the search ring until we find an empty slot
    while True:
        # try above first
        if not any(abs(up_y   - py) < MIN_SEP_Y and abs(x-px) < MIN_SEP_X for px, py in placed_xy):
            placed_xy.append((x, up_y))
            return up_y
        # then one step below
        if not any(abs(down_y - py) < MIN_SEP_Y and abs(x-px) < MIN_SEP_X for px, py in placed_xy):
            placed_xy.append((x, down_y))
            return down_y
        up_y   += LABEL_PAD
        down_y -= LABEL_PAD


# Loop over the s₂ values – here using withexplanation values 0, 1, 2
for ax, sel in zip(axes, [0, 1, 2]):
    # First, plot the original metrics
    for metric in compared_metrics2:
        df = df_map[metric].copy()
        df['conf_bin'] = pd.cut(
        df['confidence'],
        bins=bin_edges,
        include_lowest=True,
        right=True
        )
        df0 = df[df['withexplanation'] == sel]
        grouped = (df0.groupby('conf_bin', observed=False)
            .apply(lambda g: bin_stats(g, sel))
            .reset_index())
        grouped = grouped.sort_values(by='conf_bin')
        grouped['p'] *= 100
        grouped['se'] *= 100

        valid = grouped['n'] > 0
        ax.errorbar(
            bin_centers[valid],
            grouped.loc[valid, 'p'],
            yerr=grouped.loc[valid, 'se'],
            marker='o',
            linestyle='-',
            color=compared_metric_colors.get(metric, None),
            capsize=5,
            label=label_map[metric]
        )
        
        # annotate counts
        # ── 2. inside your loop that writes the labels ─────────────────────────────────
        for x, height, count in zip(bin_centers, grouped['p'], grouped['n']):
            if count == 0:
                continue
            # first decide the nominal y you would have used
            desired_y = height
            if height > 95:
                desired_y = height - 5
            elif height < 5:
                desired_y = height + 5

            
            # ask the helper for a collision‑free y position
            y_for_label = find_free_y(x + 0.08, desired_y)
            
            ax.text(
                x + 0.08,
                y_for_label,
                f'n={int(count)}',
                ha='center',
                va='bottom',
                fontsize='small',
                color=compared_metric_colors.get(metric, 'black')
            )

            
    # Set subplot titles and labels
    title = f"s{sel}: "
    if sel == 0:
        title += "User thinks AI is Correct"
    elif sel == 1:
        title += "User thinks AI is Incorrect"
    elif sel == 2:
        title += "User is Unsure"
    ax.set_title(title)
    ax.set_xlabel('Confidence')
    ax.set_xlim(0, 1)
    # ax.legend(fontsize='small')
    
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, ncol=3, fontsize='small')
axes[0].set_ylabel('Switch Rate at Stage 3 [%]', fontsize=14)
axes[0].set_ylim(0, 100) 
plt.suptitle("After seeing explanation...") 
plt.tight_layout(rect=[0, 0, 1, 1]) 
plt.show()